## Pré-Processamento

Esse notebook contém o pré-processamento de 3 tabelas (Matches, Players e Teams) e, por fim, a união delas.

**Tutorial de como rodar esse notebook:**

1 - As bibliotecas necessárias já estão sendo importadas no código, mas se, por algum motivo, não funcionar, instale as bibliotecas no seu computador através do comando **pip install (nome da biblioteca)**

2 - Ter o Python instalado

In [1]:
# Importando as bibliotecas
import pandas as pd
import matplotlib as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


Carrega as tabelas

In [2]:
def download_file_from_google_drive(file_id, destination):
    URL = "https://drive.google.com/uc?id=1NH39rh-7pBVQQU28RtrbG8fgjRR1jXbN&export=download"


In [3]:
def download_file_from_google_drive(file_id, destination):
    URL = "https://drive.google.com/uc?id=1uVRFcDySx3-J0YNsn-jsm7xv84CT1ARG&export=download"

In [4]:
def download_file_from_google_drive(file_id, destination):
    URL = "https://drive.google.com/uc?id=1xOalsah4MwE0-Ib-f4Bk2H6haPwfTXGi&export=download"

In [5]:
# Carregando o arquivo colunas-matches
dfPlayers = pd.read_csv('https://drive.google.com/uc?id=1NH39rh-7pBVQQU28RtrbG8fgjRR1jXbN&export=download')
dfTeams = pd.read_csv('https://drive.google.com/uc?id=1uVRFcDySx3-J0YNsn-jsm7xv84CT1ARG&export=download')
dfMatches = pd.read_csv('https://drive.google.com/uc?id=1xOalsah4MwE0-Ib-f4Bk2H6haPwfTXGi&export=download', delimiter=';')

# Tratamento de dados

Limpeza de dados nulos

In [ ]:
# Definir a porcentagem mínima de valores não-nulos (30% de NaN significa 70% de valores ausentes)
threshold = int(dfPlayers.shape[1] * 0.3)

# Remover as linhas que têm mais de 70% de valores NaN
dfPlayers = dfPlayers.dropna(thresh=threshold)

# Exibir o DataFrame final
print(dfPlayers)

In [7]:
# Excluir colunas que são 100% iguais a 0
dfPlayers = dfPlayers.loc[:, ~(dfPlayers == 0).all()] 

Escolhendo apenas os jogos que já aconteceram

In [8]:
# Filtrando o DataFrame para manter apenas as linhas onde o status é 'completed'
dfMatches = dfMatches[dfMatches['status'] == 'complete']

**Codificação das variáveis categóricas**

In [9]:
#Transformando coluna categórica em uma coluna numérica
label_encoder = LabelEncoder()
dfPlayers['full_name_encoded'] = label_encoder.fit_transform(dfPlayers['full_name'])

In [ ]:
#mapeando numeração de cada time
mapeamento_players = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(mapeamento_players)

In [11]:
#Transformando coluna categórica em uma coluna numérica
label_encoder = LabelEncoder()
dfTeams['common_name_encoded'] = label_encoder.fit_transform(dfTeams['common_name'])

In [ ]:
#mapeando numeração de cada time
mapeamento_times = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(mapeamento_times)

Mapeamento das variáveis númericas para que entendamos o resultado.

In [ ]:
# Dicionário de mapeamento para os times
team_mapping = {
    'Atlético GO': 0, 'Atlético Mineiro': 1, 'Atlético PR': 2, 'Bahia': 3, 'Botafogo': 4,
    'Bragantino': 5, 'Corinthians': 6, 'Criciúma': 7, 'Cruzeiro': 8, 'Cuiabá': 9,
    'Flamengo': 10, 'Fluminense': 11, 'Fortaleza': 12, 'Grêmio': 13, 'Internacional': 14,
    'Juventude': 15, 'Palmeiras': 16, 'São Paulo': 17, 'Vasco da Gama': 18, 'Vitória': 19
}

# Aplicando o mapeamento às colunas 'home_team_name' e 'away_team_name'
dfMatches['home_team_name'] = dfMatches['home_team_name'].replace(team_mapping)
dfMatches['away_team_name'] = dfMatches['away_team_name'].replace(team_mapping)

# Exibindo as primeiras linhas do DataFrame após a transformação
dfMatches.head()

In [14]:
#Transformando coluna categórica em uma coluna numérica
label_encoder = LabelEncoder()
dfPlayers['nationality_encoded'] = label_encoder.fit_transform(dfPlayers['nationality'])

In [ ]:
#mapeando numeração de cada time
mapeamento_nationality = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(mapeamento_nationality)

In [16]:
# Excluir colunas que são 100% iguais a 0
dfMatches = dfMatches.loc[:, ~(dfMatches == 0).all()] 

Remoção de colunas já tratadas e salvas em novas colunas ou colunas definidas como não úteis

In [17]:
dfPlayers = dfPlayers.drop(['nationality','Current Club', 'position', 'full_name', 'birthday'], axis=1)

In [18]:
# Excluir colunas que são 100% iguais a 0
dfPlayers = dfPlayers.loc[:, ~(dfPlayers == 0).all()] 

Tratamento dos dados team_goal_timings. Essa coluna representa o minuto em que os gols dos time aconteceram, mas tínhamos problemas com jogos que acabaram sem gols

In [ ]:
# Função para pegar o minuto do primeiro gol
def primeiro_gol(minutos):
    if pd.isna(minutos):
        return np.nan  # Mantém NaN se os minutos forem nulos

    # Divide a string em uma lista, ignorando espaços e apostrofos
    minutos = [m.split("'")[0].strip() for m in str(minutos).split(',') if m]

    # Filtra apenas valores que podem ser convertidos para inteiros
    try:
        minutos = [int(m) for m in minutos if m.isdigit()]
    except ValueError:
        return np.nan  # Retorna NaN se houver erro na conversão
    
    if not minutos:
        return np.nan  # Retorna NaN se a lista estiver vazia
    
    # Retorna o primeiro valor da lista ordenada
    return sorted(minutos)[0]

# Aplicando a função para o primeiro gol
dfMatches['home_team_first_goal'] = dfMatches['home_team_goal_timings'].apply(primeiro_gol)
dfMatches['away_team_first_goal'] = dfMatches['away_team_goal_timings'].apply(primeiro_gol)

# Exibindo as primeiras linhas
dfMatches[['home_team_goal_timings', 'home_team_first_goal', 'away_team_goal_timings', 'away_team_first_goal']].head(20)

In [20]:
dfMatches = dfMatches.drop(['away_team_goal_timings', 'home_team_goal_timings'], axis=1)

# Gráficos de importancia das colunas

In [ ]:
# Selecionar as colunas de interesse (Ex: quem marca o placar final)
target_columns = ['home_team_goal_count', 'away_team_goal_count']

# Separar as features (X) dos alvos (y)
X = dfMatches.drop(target_columns, axis=1)
y = dfMatches[target_columns]

# Remover variáveis categóricas ou convertê-las em numéricas (se ainda não foi feito)
X = X.select_dtypes(exclude=['object'])

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X.fillna(0), y.fillna(0), test_size=0.3, random_state=42)

# 3. Utilizar RandomForest para ver a importância das features
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Obter as importâncias das features
importances = model.feature_importances_
features = X.columns

# Criar um DataFrame com as importâncias
feature_importances = pd.DataFrame({'Feature': features, 'Importance': importances})

# Ordenar as features pela importância, da maior para a menor
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Exibir as 20 variáveis que mais influenciam no modelo
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importances.head(20))  # Mostrar as 20 mais importantes
plt.title('Top 20 Features mais Importantes')
plt.show()
print(feature_importances)

In [ ]:
#Selecionar as colunas de interesse (Ex: quem marca o placar final)
target_columns = ['home_team_goal_count', 'away_team_goal_count']

# Separar as features (X) dos alvos (y)
X = dfMatches.drop(target_columns, axis=1)
y = dfMatches[target_columns]

# Remover variáveis categóricas ou convertê-las em numéricas (se ainda não foi feito)
X = X.select_dtypes(exclude=['object'])

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X.fillna(0), y.fillna(0), test_size=0.3, random_state=42)

# 3. Utilizar RandomForest para ver a importância das features
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Obter as importâncias das features
importances = model.feature_importances_
features = X.columns

# Criar um DataFrame com as importâncias
feature_importances = pd.DataFrame({'Feature': features, 'Importance': importances})

# Ordenar as features pela importância, da menor para a maior
feature_importances = feature_importances.sort_values(by='Importance', ascending=True)

# Exibir as 20 variáveis que menos influenciam no modelo (ou seja, as que mais atrapalham)
plt.figure(figsize=(10, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importances.head(20))  # Mostrar as 20 menos importantes
plt.title('Top 20 Features Menos Importantes (Mais Atrapalham)')
plt.show()


A partir dos gráficos acima, obtém-se as variáveis que mais impactam no modelo, tendo o nome de sua coluna e seu respecitivo grau de importância. Dessa forma, é possível utilizar as mais importantes dentro da aplicação que está sendo desenvolvida

Gera o csv das colunas tratadas para serem utilizadas em novos notebooks

In [23]:
dfPlayers.to_csv('dfPlayers.csv', index=False)
dfMatches.to_csv('dfMatches.csv', index=False)
dfTeams.to_csv('dfTeams.csv', index=False)